In [1]:
# !pip install transformers accelerate jsonformer

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

print("Loading model and tokenizer...")
model_name = "ibm-granite/granite-3.1-2b-instruct"
model = AutoModelForCausalLM.from_pretrained(model_name, use_cache=True, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, use_cache=True)
print("Loaded model and tokenizer")

# Enable mixed precision
model = model.half()  # Convert model to half precision

Loading model and tokenizer...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded model and tokenizer


In [5]:
from jsonformer.format import highlight_values
from jsonformer.main import Jsonformer

invoice_schema = {
    "type": "object",
    "properties": {
        "line_items": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "product_code": {"type": "string"},
                    "description": {"type": "string"},
                    "quantity": {"type": "string"},
                    "price_per_unit": {"type": "string"},
                    "vat_percent": {"type": "string"},
                    "total_price": {"type": "string"}
                }
            }
        },
        },
    }

builder = Jsonformer(
    model=model,
    tokenizer=tokenizer,
    json_schema=invoice_schema,
    prompt="generate array of items from the text : 'Invoice To:\nDirect Produce Supplies LTD\n57-63 Church Street\nWimbledon\nSW19 5DQ\nInvoice Date: 22/11/2023\nInvoice No:22806\nQuantity\nRate (£)\nNet Value (£)\n178,082\n0.090\n10,641.96\n105,084\n0.090\n2,856.15\n36,065\n0.090\n1,762.74\nNet Total (£):\n15,260.85\nVAT (£):\n3,052.17\nBank Details: HSBC UK Bank Plc\nPayment Terms: 28 days from invoice date\nGross Total (£):\n18,313.02\nSort Code: 40-03-21\nAccount No: 2236 6789\nContact\n T 01795 523 310\n E accounts@issproduce.co.uk\nW www.issproduce.co.uk\nINTEGRATED SERVICE SOLUTIONS\nIntegrated Service Solutions Limited, C/O Fowler Welch, London Road, Teynham, Sittingbourne, ME9 9PR\nService Description\nISS SP - Picking: 12/11/2023 - 18/11/2023 Teynham\nISS SP - Picking: 12/11/2023 - 18/11/2023 Linton\nISS SP - Picking: 12/11/2023 - 18/11/2023 Sittingbourne\nVAT Registration No: 184 6959 49            Company Registration No: 08332191\n'")

print("Generating...")
output = builder()

highlight_values(output)


Generating...
{
  line_items: [
    {
      product_code: "178082",
      description: "178,082",
      quantity: "1",
      price_per_unit: "0.090",
      vat_percent: "0.090",
      total_price: "10.64196"
    }
  ]
}
